In [2]:
from IPython.display import Image

# MQs Throughput
0. 요약
1. 카프카 성능 테스트
    1. AWS에 카프카 클러스터 구성
    2. 카프카 성능 테스트 (단일/다중 프로듀서)

2. 래빗엠큐 성능 테스트
    1. AWS에 레빗엠큐 클러스터 구성
    2. 레빗엠큐 성능 테스트 (단일 프로듀서)

부록: 리눅스 리소스 상태 관련 명령어

---

## 0. 요약
![per_compar1](./assets/kafka_vs_rabbitmq_1.png)  
![per_compar2](./assets/kafka_vs_rabbitmq_2.png)  
([출처: Kafka: a Distributed Messaging System for Log Processing, Jay Kreps, Linkedin, 2011](http://notes.stephenholiday.com/Kafka.pdf))  
- 실제 AWS 에서 기본 설정값으로 테스트시 위 그림처럼 `Kafka` 와 `Rabbitmq` 의 차이가 수십배 정도 차이가 나지 않았다.  

- `kafka` 에서 단일 프로듀서 사용시 `60MB/Sec`, `2KB 레코드 초당 3만개`, 프로듀서 2개 사용시 `160MB/sec`, `2Kb 레코드 초당 6~8만개` 처리 성능을 보였다.  

- `rabbitmq` 에서 단일 프로듀서 사용시 `20MB/Sec`, `2KB 레코드 초당 1만개` 처리 성능을 보였다.  

- 둘다 같은 인스턴스 스펙과 HDD persistence logging 을 사용했고 프로듀서 환경도 최대한 비슷하게 구성하였다.  

> kafka 성능에 영향을 주는 파라미터는 약 20개 정도 존재하며 실제 프로덕션에서 제약조건에 따라 파라미터 튜닝을 통해 최대 네트워크 대역폭까지 사용할 수 있을 것으로 예상된다.  
> 파라미터 성능 튜닝 내용은[여기](https://www.slideshare.net/freepsw/apache-kafka-metrics-123663954)를 참조한다.

---

## 1. 카프카 성능 테스트 구성

### 1. 1. AWS

#### 인스턴스, 네트워크 설정

- AMI
    1. Kafka Broker, Zookeeper
        - AMI: Ubuntu Server 18.04 LTS (HVM), ami-0794a2d1e6d99117a
        - Kafka Version: kafka_2.12-2.2.0
        - jdk version: openjdk version "1.8.0_212"
    3. Producer/Consumer
        - AMI: Ubuntu Server 18.04 LTS (HVM), ami-0794a2d1e6d99117a
        - library version: confluent_kafka==1.0.1

- Instance Type: 
    1. Kafka Broker (i2.xlarge * 3대)
        - vCPU: 4
        - Memory: 30.5GB
        - SSD: 800GB (EBS, General Purpose SSD)
        - Network: Moderate
        
    2. Zookeeper
        - vCPU: 4
        - Memory: 30.5GB
        - SSD: 800GB (EBS)
        - Network: Moderate
        > 주키퍼는 카프카 설치시 포함됨으로 1,2,3번 카프카 브로커 내부에 주키퍼 노드와 카프카 브로커를 동시에 띄워서 테스트했다.
        
    3. Producer (c5.large * 2대)
        - vCPU: 2
        - Memory: 4
        - Network: Up to 10 Gigabit

- network configuration
    모든 인스턴스는 같은 VPC내의 Subnet에 위치되었다.
    - Kafka
        1. broker1: 172.31.9.217
        2. broker2: 172.31.11.158
        3. broker3: 172.31.8.224
    - Zookeeper
        1. broker1: 172.31.9.217
        2. broker2: 172.31.11.158
        3. broker3: 172.31.8.224
    - Producer
        1. Producer1: 172.31.3.227
        2. Producer2: 172.31.6.86

### Kafka 클러스터 설치

모든 브로커(broker 1, 2, 3)에 아래와 같은 작업(1., 2.)을 공통적으로 수행한다.

##### 1. jdk 설치
```Bash
add-apt-repository ppa:openjdk-r/ppa
apt-get update 
apt-get install openjdk-8-jdk -y
java -version
openjdk version "1.8.0_212"
OpenJDK Runtime Environment (build 1.8.0_212-8u212-b03-0ubuntu1.18.04.1-b03)
OpenJDK 64-Bit Server VM (build 25.212-b03, mixed mode)
```

##### 2. 카프카 다운로드
[download](https://www.apache.org/dyn/closer.cgi?path=/kafka/2.2.0/kafka_2.12-2.2.0.tgz) 미러사이트 에서 다운로드한다.
> 예제에서는 `/home/ubuntu` 에 다운로드하고 설치하지만 보안을 고려한다면 다른 경로를 사용한다.  
> 카프카 다운로드 시 주키퍼가 포함되어 있다.

```Bash
cd ~
pwd
/home/ubuntu

wget http://mirror.navercorp.com/apache/kafka/2.2.0/kafka_2.12-2.2.0.tgz 
tar -zxf kafka_2.12-2.2.0.tgz 
cd kafka_2.12-2.2.0
```

각 브로커 서버(broker 1,2,3)에서 주키퍼 노드를 띄울 것이므로 아래와 같은 작업(3., 4.)을 공통적으로 수행한다.
##### 3. 주키퍼 설정
```Bash
(root) $ pwd
/home/ubuntu/kafka_2.12-2.2.0
(root) $ cd config
(root) $ nano zookeeper.properties
```

```Properties
dataDir=/tmp/zookeeper
clientPort=2181
maxClientCnxns=0
initLimit=5
syncLimit=2
server.1=172.31.9.217:2888:3888
server.2=172.31.11.158:2888:3888
server.3=172.31.8.224:2888:3888
```

4. 주키퍼 myid 생성 (이 과정은 각 broker 서버에서 작업을 수행한다.)
    1. broker1(172.31.9.217)
        ```Bash
        mkdir /tmp/zookeeper
        echo 1 > /tmp/zookeeper/myid
        ```

    2. broker2(172.31.11.158)
        ```Bash
        mkdir /tmp/zookeeper
        echo 2 > /tmp/zookeeper/myid
        ```

    3. broker3(172.31.8.224)
        ```Bash
        mkdir /tmp/zookeeper
        echo 3 > /tmp/zookeeper/myid
        ```

각 브로커 서버(broker 1,2,3)에서 주키퍼 노드를 작동시킨다.  
5. 주키퍼 노드 시작
```Bash
cd /home/ubuntu/kafka_2.12-2.2.0
bin/zookeeper-server-start.sh config/zookeeper.properties
```

각 브로커 서버들에서 주키퍼 노드들이 정상적으로 작동하였으면 주키퍼를 데몬으로 띄운다.
6. 주키퍼 데몬 시작
```Bash
bin/zookeeper-server-start.sh -daemon config/zookeeper.properties
```
> `netstat -lntp` 명령어로 주키퍼 노드(`2181 포트`)가 살아있는지 확인 할 수 있다. 

각 브로커 서버(broker1, 2, 3)에서 아래와 같은 작업(7.)을 각각 수행한다.
7. 카프카 브로커 설정  
`i2.xlarge` 인스턴스는 800GB SSD 스토리지를 사용할 수 있다.  
여기에 카프카 로그 세그먼테이션을 저장하도록 설정한다.
> 이 스토리지는 초기화 되지 않았음으로 아래 명령어를 통해 초기화 과정을 거친다.

    1. `df -h` 명령어로 SSD 스토리지가 아직 마운트 되지 않음을 확인한다.
    2. `fdisk -l` 명령어로 SSD 스토리지 드라이버가 부착 되있고 경로는 `/dev/xvdb` 임을 확인한다.
    3. `mkfs.ext4 -E nodiscard /dev/xvdb` 명령어로 파일 시스템을 생성한다.
    4. `mkdir /mnt/aws_ssd` 명령어로 해당 스토리지가 마운트될 폴더를 생성한다.
    5. `sudo mount -o discard /dev/xvdb /mnt/aws_ssd` 명령어로 해당 스토리지를 마운트한다.
    6. `mkdir /mnt/aws_ssd/kafka_logs` 명령어로 카프카 로그가 저장될 디렉토리를 생성한다.
    
```Bash
df -h
Filesystem      Size  Used Avail Use% Mounted on
udev             15G     0   15G   0% /dev
tmpfs           3.0G  772K  3.0G   1% /run
/dev/xvda1      7.7G  1.8G  6.0G  23% /
tmpfs            15G     0   15G   0% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
tmpfs            15G     0   15G   0% /sys/fs/cgroup
/dev/loop0       89M   89M     0 100% /snap/core/7169
/dev/loop1       18M   18M     0 100% /snap/amazon-ssm-agent/1335
tmpfs           3.0G     0  3.0G   0% /run/user/1000

fdisk -l
# ...
Disk /dev/xvda: 8 GiB, 8589934592 bytes, 16777216 sectors
Units: sectors of 1 * 512 = 512 bytes
Sector size (logical/physical): 512 bytes / 512 bytes
I/O size (minimum/optimal): 512 bytes / 512 bytes
Disklabel type: dos
Disk identifier: 0x34283f9c

Device     Boot Start      End  Sectors Size Id Type
/dev/xvda1 *     2048 16777182 16775135   8G 83 Linux

Disk /dev/xvdb: 745.2 GiB, 800165027840 bytes, 1562822320 sectors
Units: sectors of 1 * 512 = 512 bytes
Sector size (logical/physical): 512 bytes / 4096 bytes
I/O size (minimum/optimal): 4096 bytes / 4096 bytes

mkfs.ext4 -E nodiscard /dev/xvdb
mkdir /mnt/my_aws_ssd
mount -o discard /dev/xvdb /mnt/my_aws_ssd
```

마운트된 스토리지를 확인한다.

```Bash
df -h
# ...
tmpfs           3.0G     0  3.0G   0% /run/user/1000
/dev/xvdb       733G   73M  696G   1% /mnt/aws_ssd
```

```Bash
cd config
nano server.properties
```  

각 브로커마다 설정 파일(`server.properties`)을 아래와 같이 변경한다.
- Broker1 (172.31.9.217)
```Properties
############################# Server Basics ############################
broker.id=1
############################# Socket Server Settings #############################
# ...
advertised.listeners=PLAINTEXT://172.31.9.217:9092
# ...
############################# Log Basics #############################
# ...
log.dirs=/mnt/aws_ssd/kafka_logs
# ...
############################# Zookeeper ##############################
zookeeper.connect=172.31.9.217:2181,172.31.11.158:2181,172.31.8.224:2181
# ...
```

- Broker2 (172.31.11.158)
```Properties
############################# Server Basics ############################
broker.id=2
############################# Socket Server Settings #############################
# ...
advertised.listeners=PLAINTEXT://172.31.11.158:9092
# ...
############################# Log Basics #############################
# ...
log.dirs=/mnt/aws_ssd/kafka_logs
# ...
############################# Zookeeper ##############################
zookeeper.connect=172.31.9.217:2181,172.31.11.158:2181,172.31.8.224:2181
# ...
```


- Broker3 (172.31.8.224)
```Properties
############################# Server Basics ############################
broker.id=3
############################# Socket Server Settings #############################
# ...
advertised.listeners=PLAINTEXT://172.31.8.224:9092
# ...
############################# Log Basics #############################
# ...
log.dirs=/mnt/aws_ssd/kafka_logs
# ...
############################# Zookeeper ##############################
zookeeper.connect=172.31.9.217:2181,172.31.11.158:2181,172.31.8.224:2181
# ...
```

각 브로커마다 `bin/kafka-run-class.sh` 파일을 아래와 같이 변경한다.
- Broker1 (172.31.9.217)
```Bash
# ...
# JMX settings
if [ -z "$KAFKA_JMX_OPTS" ]; then
  KAFKA_JMX_OPTS="-Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Djava.rmi.server.hostname=172.31.9.217 -Djava.net.preferIPv4Stack=true"
fi
# ...
```

- Broker2 (172.31.11.158)
```Bash
# ...
# JMX settings
if [ -z "$KAFKA_JMX_OPTS" ]; then
  KAFKA_JMX_OPTS="-Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Djava.rmi.server.hostname=172.31.11.158 -Djava.net.preferIPv4Stack=true"
fi
# ...
```

- Broker3 (172.31.8.224)
```Bash
# ...
# JMX settings
if [ -z "$KAFKA_JMX_OPTS" ]; then
  KAFKA_JMX_OPTS="-Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Djava.rmi.server.hostname=172.31.8.224 -Djava.net.preferIPv4Stack=true"
fi
# ...
```

모든 브로커에서 `bin/kafka-server-start.sh` 파일 최상단에 아래와 같은 명령어를 추가한다. 
- Broker 1,2,3
```Bash
# ...
export JMX_PORT=${JMX_PORT:-9999}
# ...
```
> `JMX_PORT` 변수는 JMX(Java Management Extension) 포트이고 모니터링시 `kafka-manager` 에 메트릭을 전송한다.


설정 파일 수정 후 카프카 브로커를 각각 서버에서 띄운다.
8. `Broker1, 2, 3` 에서 카프카 브로커 시작
```Bash
bin/kafka-server-start.sh config/server.properties
```

카프카 브로커가 정상 작동시 데몬으로 띄운다.
9. `Broker1, 2, 3` 에서 카프카 브로커 데몬 시작
```Bash
bin/kafka-server-start.sh -daemon config/server.properties
```

카프카 브로커의 성능 메트릭을 확인하기 위해 모니터링 툴(`kafka-manager`)을 설치한다.  
> 아래 작업(10., 11., 12., 13.)은 broker1 (172.31.9.217) 에서만 실행한다.

10. 모니터링 툴 다운로드 및 빌드
```Bash
git clone https://github.com/yahoo/kafka-manager
cd kafka-manager
./sbt clean dist
```
> 이 빌드 작업은 i2.xlarge 기준 약 15분 정도 걸린다.

11. 빌드파일 확인
```Bash
cd target/universal
unzip kafka-manager-2.0.0.2.zip
```

12. `kafka-manager` 설정 변경
```Bash
vi conf/application.conf
```

```Properties
kafka-manager.zkhosts="172.31.9.217:2181,172.31.11.158:2181,172.31.8.224:2181"
basicAuthentication.username="admin"
basicAuthentication.password="test"
```

13. 카프카 매니저 실행
```Bash
bin/kafka-manager
```

카프카 매니저는 `broker1_public_ip:9000` 로 접속할 수 있다.   
접속 후 `Add Cluster` 버튼을 눌러 아래와 같이 클러스터를 추가한뒤 클러스터 상태를 모니터링 할 수 있다.  
![kafka-eos](./assets/kafka_manager.png)

카프카 브로커 정상 작동을 확인하기 위해 broker1(172.31.9.217)에서 메시지를 프로듀싱하고, broker2(172.31.11.158)에서 컨슈밍테스트를 한다.
- Broker1(172.31.9.217)
```Bash
bin/kafka-console-producer.sh --broker-list 172.31.9.217:9092 --topic test0
> 1
> 2
> 3
> ^C
```

- Broker2(172.31.11.158)
```Bash
bin/kafka-console-consumer.sh --bootstrap-server 172.31.9.217:9092 --offset earliest --partition 0 --topic test0
1
2
3
^CProcessed a total of 3 messages
```

---

### 1. 2. 카프카 성능 테스트

프로듀서 노드(`Producer1`: 172.31.3.227)에서 아래와 같은 작업을 수행한다.

1. 카프카 성능 테스트 쉘 스크립트 다운로드
```Bash
wget http://mirror.navercorp.com/apache/kafka/2.2.0/kafka_2.12-2.2.0.tgz 
tar -zxf kafka_2.12-2.2.0.tgz 
```
> 위 스크립트는 카프카 공식 릴리즈에서 사용하는 테스트 스크립트이다. 

2. 카프카 관리 쉘 스크립트를 통한 (**단일 프로듀서**) 성능테스트
> `Producer1`(172.31.3.227) 에서 실행

토픽생성
```Bash
bin/kafka-topics.sh --create --zookeeper 172.31.9.217:2181,172.31.11.158:2181,172.31.8.224:2181 --replication-factor 3 --partitions 3 --topic test_topic3
```

프로듀싱
```Bash
# 2kbyte 레코드를 날릴 경우 초당 평균 33783개 레코드를 프로듀싱 할 수 있다.
bin/kafka-producer-perf-test.sh --topic test_topic3 --throughput 100000 --record-size 2000 --num-records 2000000 --producer-props bootstrap.servers=172.31.9.217:9092,172.31.11.158:9092,172.31.8.224:9092 acks=1

66785 records sent, 13357.0 records/sec (25.48 MB/sec), 901.9 ms avg latency, 1407.0 ms max latency.  
73976 records sent, 14795.2 records/sec (28.22 MB/sec), 1084.8 ms avg latency, 1916.0 ms max latency.  
160528 records sent, 32105.6 records/sec (61.24 MB/sec), 530.4 ms avg latency, 1506.0 ms max latency.  
240749 records sent, 48149.8 records/sec (91.84 MB/sec), 331.7 ms avg latency, 1108.0 ms max latency.  
228682 records sent, 45736.4 records/sec (87.24 MB/sec), 370.1 ms avg latency, 1037.0 ms max latency.  
181547 records sent, 36309.4 records/sec (69.25 MB/sec), 451.1 ms avg latency, 703.0 ms max latency.  
183616 records sent, 36723.2 records/sec (70.04 MB/sec), 444.9 ms avg latency, 727.0 ms max latency.  
180200 records sent, 36040.0 records/sec (68.74 MB/sec), 454.8 ms avg latency, 748.0 ms max latency.  
182600 records sent, 36520.0 records/sec (69.66 MB/sec), 447.6 ms avg latency, 756.0 ms max latency.  
181357 records sent, 36264.1 records/sec (69.17 MB/sec), 451.9 ms avg latency, 754.0 ms max latency.  
179995 records sent, 35999.0 records/sec (68.66 MB/sec), 454.3 ms avg latency, 817.0 ms max latency.  
2000000 records sent, 33783.213121 records/sec (64.44 MB/sec), 474.29 ms avg latency, 1916.00 ms max latency, 614 ms 50th, 1092 ms 95th, 1445 ms 99th, 1825 ms 99.9th.
```
> 단일 프로듀서 사용시 `64.44MB/Sec` 퍼포먼스를 보인다.

컨슈밍
```Bash
bin/kafka-consumer-perf-test.sh --broker-list 172.31.8.224:9092 --messages 500000 --topic test_topic3 --threads 10

start.time, end.time, data.consumed.in.MB, MB.sec, data.consumed.in.nMsg, nMsg.sec, rebalance.time.ms, fetch.time.ms, fetch.MB.sec, fetch.nMsg.sec  
2019-07-04 05:50:08:683, 2019-07-04 05:50:11:322, 953.7868, 361.4198, 500059, 189488.0637, 17, 2622, 363.7631, 190716.6285  
```

----

1. 카프카 성능 테스트 파이썬 스크립트 다운로드
```Bash
git clone https://github.com/bohblue2/kafka-rabbitmq-test
cd kafka-rabbitmq-test
pip3 install -r requirements.txt
```

2. 카프카 성능 테스트 파이썬 스크립트를 통한 (다중 프로듀서) 성능 테스트
> `Producer1`, `Producer2` 에서 동시에 실행 후 카프카 모니터링 툴에서 Throughput 확인

토픽생성
```Bash
bin/kafka-topics.sh --create --zookeeper 172.31.9.217:2181,172.31.11.158:2181,172.31.8.224:2181  --replication-factor 3 --partitions 3 --topic test_topic5
```

프로듀싱
- `Producer1`(172.31.3.227)
```Bash
python3 test_aws_kafka_producer.py --tests 1 --topic test_topic5 --brokers 172.31.9.217:2181,172.31.11.158:2181,172.31.8.224:2181 --idempotence false --partitions 3 --msg-count 10000000 --producers 1 --data-size 2 --acks-mode 1
```

- `Producer2`(172.31.6.86)
```Bash
python3 test_aws_kafka_producer.py --tests 1 --topic test_topic5 --brokers 172.31.9.217:2181,172.31.11.158:2181,172.31.8.224:2181 --idempotence false --partitions 3 --msg-count 10000000 --producers 1 --data-size 2 --acks-mode 1
```

결과  
![kafka_manager3](./assets/kafka_manager_3.png)
> 다중 프로듀서 사용시 `초당 8만개 레코드`, `160MB/Sec` 퍼포먼스를 보여주고 있다.  
> 한 레코드 크기는 `2KB`, `acks는 1` 로 프로듀서를 설정하였다.

---

## 2. Rabbitmq 성능 테스트 구성

### 2. 1. AWS

#### 인스턴스, 네트워크 설정

- AMI
    1. Rabbitmq Broker
        - AMI: Ubuntu Server 18.04 LTS (HVM), ami-0794a2d1e6d99117a
        - Rabbitmq Version: 3.7.15

    3. Producer/Consumer
        - AMI: Ubuntu Server 18.04 LTS (HVM), ami-0794a2d1e6d99117a
        - library version: pika
        - python version: 3.7

- Instance Type: 
    1. Rabbitmq Broker (i2.xlarge * 3대)
        - vCPU: 4
        - Memory: 30.5GB
        - SSD: 800GB (EBS, General Purpose SSD)
        - Network: Moderate
        
    3. Producer (c5.large * 2대)
        - vCPU: 2
        - Memory: 4
        - Network: Up to 10 Gigabit

- network configuration
    모든 인스턴스는 같은 VPC내의 Subnet에 위치되었다.
    - Rabbitmq
        1. broker1: 172.31.6.112
        2. broker2: 172.31.0.43
        3. broker3: 172.31.5.61
    - Producer
        1. Producer1: 172.31.3.227
        2. Producer2: 172.31.6.86

#### Rabbitmq 클러스터 설치

모든 브로커(`broker1, 2, 3`)에서 아래와 같은 작업을 각각 수행한다.

1. rabbitmq server 설치
```Bash
sudo apt-get install rabbitmq-server -y
# 서버 상태 확인
sudo rabbitmqctl cluster_status
```

2. earlang cookie 복사
> rabbitmq 클러스터 구성시 모든 노드들의 `earlang cookie` 값이 모두 같아야 node auth가 된다.

`broker1`(172.31.6.112)에서 아래 명령어를 통해 earlang cookie 값을 얻어서 `broker2`, `broker3` 의 earlang cookie 값을 `broker1` 과 같게 수정한다.

```Bash
# broker1(172.31.6.122)
sudo cat /var/lib/rabbitmq/.erlang.cookie

DDJJOJQAUNEEXOBIJFBK
```

```Bash
# broker2(172.31.0.43)
sudo -s
echo DDJJOJQAUNEEXOBIJFBK > /var/lib/rabbitmq/.erlang.cookie
```

```Bash
# broker3(172.31.5.61)
sudo -s
echo DDJJOJQAUNEEXOBIJFBK > /var/lib/rabbitmq/.erlang.cookie
```

3. 클러스터를 구성하기 위해 모든 브로커들의 서버를 재시작 시킨다.
```Bash
sudo service rabbitmq-server restart
sudo rabbitmqctl stop_app
sudo rabbitmqctl reset
```

4. 모든 브로커들의 `/etc/hosts` 파일을 아래와 같이 수정
> rabbitmq 클러스터들의 노드들이 `rabbit@ip-XXX-XXX-XX-XX` 형식으로 네이밍되기에 `/etc/hosts` 파일을 아래와 같이 변경한다.

```
sudo nano /etc/hosts
```

```
127.0.0.1 localhost
172.31.6.122 ip-172-31-6-122
172.31.0.43 ip-172-31-0-43
172.31.5.61 ip-172-31-5-61
```

5. `broker2`, `broker3` 서버에서 아래의 명령어를 수행하여 `broker1` 의 클러스터로 조인한다. 
```Bash
sudo rabbitmqctl join_cluster rabbit@ip-172-31-6-112
```
> Error:  Mnesia is still running on node 'rabbit@ip-172-31-0-43'.  
Please stop the node with rabbitmqctl stop_app first.  
오류 발생시 `sudo rabbitmqctl stop_app` 명령어를 수행한다.


6. `braoker1` 에서 아래 명령어를 수행하여 rabbitmq 계정 생성.
> 외부에서 rabbitmq 사용시(메시지 퍼블리쉬 등) rabbitmq 계정이 필요하다.
```Bash
sudo rabbitmqctl add_user test_admin test_password 
sudo rabbitmqctl set_user_tags test_admin administrator
sudo rabbitmqctl set_permissions -p / test_admin ".*" ".*" ".*"
```

7. 모든 브로커(`broker1`, `broker2`, `broker3`)에서 아래 명령어를 수행하여 모니터링 가능.
```Bash
sudo rabbitmqctl start_app
sudo rabbitmq-plugins enable rabbitmq_management
```

`braoker1` 의 `public_ip:15672` 로 접속하여 클러스터 상태 확인.

---

### 2.2. Rabbitmq 성능 테스트

프로듀서 노드들(`Producer1`: 172.31.3.22, `Producer2`: )에서 아래와 같은 작업을 수행한다.

1. rabbitmq 성능 테스트 툴([`Rabbitmq PerfTest`](https://rabbitmq.github.io/rabbitmq-perf-test/stable/htmlsingle/))설치

```Bash
# jdk 설치
add-apt-repository ppa:openjdk-r/ppa
apt-get update 
apt-get install openjdk-8-jdk -y

# Rabbitmq PerfTest 설치
wget https://github.com/rabbitmq/rabbitmq-perf-test/releases/download/v2.8.1/rabbitmq-perf-test-2.8.1-bin.zip
unzip rabbitmq-perf-test-2.8.1-bin.zip
cd rabbitmq-perf-test-2.8.1
```

2. rabbitmq 성능 테스트 툴을 통한 (단일 프로듀서) 성능 테스트

```Bash
bin/runjava com.rabbitmq.perf.PerfTest -y0 -p -u "throughput-test-1" -s 2000 -C 1000000 --id "test-1" -f persistent

id: test-1, starting producer #0  
id: test-1, starting producer #0, channel #0  
id: test-1, time: 1.021s, sent: 10831 msg/s  
id: test-1, time: 2.073s, sent: 15056 msg/s  
id: test-1, time: 3.145s, sent: 10113 msg/s  
id: test-1, time: 4.185s, sent: 12803 msg/s  
id: test-1, time: 5.229s, sent: 11401 msg/s  
id: test-1, time: 6.513s, sent: 6954 msg/s   
id: test-1, time: 7.549s, sent: 7717 msg/s  
id: test-1, time: 8.581s, sent: 13600 msg/s  
id: test-1, time: 9.633s, sent: 10806 msg/s  
id: test-1, time: 10.633s, sent: 14794 msg/s  
id: test-1, time: 11.797s, sent: 11212 msg/s  
id: test-1, time: 12.849s, sent: 8712 msg/s  
id: test-1, time: 13.893s, sent: 12119 msg/s  
id: test-1, time: 14.897s, sent: 12865 msg/s  
id: test-1, time: 15.969s, sent: 8426 msg/s  
```
> 위 명령어는 2KB 크기 메시지 1000000개를 (스토리지에 저장) 프로듀싱 테스트를 진행했을때 약 `20MB/Sec` 처리 성능을 보여준다.

---

#### Reference

##### 카프카 vs 레빗엠큐 성능 관련
https://hamait.tistory.com/403

https://www.infoq.com/articles/apache-kafka/

https://engineering.linkedin.com/kafka/benchmarking-apache-kafka-2-million-writes-second-three-cheap-machines

http://notes.stephenholiday.com/Kafka.pdf

##### AWS 스토리지 마운트 관련
https://docs.aws.amazon.com/ko_kr/AWSEC2/latest/UserGuide/add-instance-store-volumes.html

##### 래빗엠큐 설치관련
https://medium.com/@vipin.pratap18/rabbitmq-cluster-on-aws-ec2-with-high-availability-1bcd3f8a6404

https://medium.com/aubergine-solutions/setting-up-rabbitmq-over-ec2-instances-f4dfe49f6253

---

## 부록: 카프카 브로커 자원 사용량 확인
1. 카프카 모니터링 툴 접속
> 상용 카프카 솔루션 사용시 상세한 메트릭을 볼 수 있으나 이 예제에서는 기본 명령을 통해 확인한다.

2. 디스크 사용량 
```Bash
df -h
```

3. CPU 사용량
```Bash
# 1.
top
```

```Bash
# 2.
pidstat 1
```
    
4. 네트워크 사용량
```Bash
sar -n DEV 1
```
